# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 28 2022 11:10AM,240616,19,60017833,"GUSA Granules USA, Inc.",Released
1,Jun 28 2022 11:09AM,240615,10,MSP211635,"Methapharm, Inc.",Released
2,Jun 28 2022 11:05AM,240607,15,8199651,"Brookfield Pharmaceuticals, LLC",Released
3,Jun 28 2022 11:05AM,240607,15,8199652,"Brookfield Pharmaceuticals, LLC",Released
4,Jun 28 2022 11:05AM,240607,15,8199653,"Brookfield Pharmaceuticals, LLC",Released
5,Jun 28 2022 11:05AM,240607,15,8199654,"Brookfield Pharmaceuticals, LLC",Released
6,Jun 28 2022 11:05AM,240607,15,8199655,"Brookfield Pharmaceuticals, LLC",Released
7,Jun 28 2022 11:05AM,240607,15,8199656,"Brookfield Pharmaceuticals, LLC",Released
8,Jun 28 2022 11:05AM,240607,15,8199657,"Brookfield Pharmaceuticals, LLC",Released
9,Jun 28 2022 11:05AM,240607,15,8199658,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,240611,Released,1
32,240613,Released,1
33,240614,Released,1
34,240615,Released,1
35,240616,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240611,NaN,NaN,1.0
240613,NaN,NaN,1.0
240614,NaN,NaN,1.0
240615,NaN,NaN,1.0
240616,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240526,23.0,0.0,1.0
240534,0.0,0.0,1.0
240535,0.0,1.0,0.0
240549,0.0,0.0,1.0
240550,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240526,23,0,1
240534,0,0,1
240535,0,1,0
240549,0,0,1
240550,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240526,23,0,1
1,240534,0,0,1
2,240535,0,1,0
3,240549,0,0,1
4,240550,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240526,23,,1
1,240534,,,1
2,240535,,1,
3,240549,,,1
4,240550,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc."
1,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc."
2,Jun 28 2022 11:05AM,240607,15,"Brookfield Pharmaceuticals, LLC"
44,Jun 28 2022 11:03AM,240606,15,"Alliance Pharma, Inc."
63,Jun 28 2022 11:00AM,240614,18,"ClearSpec, LLC"
64,Jun 28 2022 10:52AM,240613,19,"GUSA Granules USA, Inc."
65,Jun 28 2022 10:26AM,240611,19,ACG North America LLC
66,Jun 28 2022 10:11AM,240610,10,"Methapharm, Inc."
71,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated"
72,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc.",,,1
1,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc.",,,1
2,Jun 28 2022 11:05AM,240607,15,"Brookfield Pharmaceuticals, LLC",,,42
3,Jun 28 2022 11:03AM,240606,15,"Alliance Pharma, Inc.",,,19
4,Jun 28 2022 11:00AM,240614,18,"ClearSpec, LLC",,,1
5,Jun 28 2022 10:52AM,240613,19,"GUSA Granules USA, Inc.",,,1
6,Jun 28 2022 10:26AM,240611,19,ACG North America LLC,,,1
7,Jun 28 2022 10:11AM,240610,10,"Methapharm, Inc.",,,5
8,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated",,,1
9,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.,,6,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc.",1,,
1,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc.",1,,
2,Jun 28 2022 11:05AM,240607,15,"Brookfield Pharmaceuticals, LLC",42,,
3,Jun 28 2022 11:03AM,240606,15,"Alliance Pharma, Inc.",19,,
4,Jun 28 2022 11:00AM,240614,18,"ClearSpec, LLC",1,,
5,Jun 28 2022 10:52AM,240613,19,"GUSA Granules USA, Inc.",1,,
6,Jun 28 2022 10:26AM,240611,19,ACG North America LLC,1,,
7,Jun 28 2022 10:11AM,240610,10,"Methapharm, Inc.",5,,
8,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated",1,,
9,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.,15,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc.",1,,
1,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc.",1,,
2,Jun 28 2022 11:05AM,240607,15,"Brookfield Pharmaceuticals, LLC",42,,
3,Jun 28 2022 11:03AM,240606,15,"Alliance Pharma, Inc.",19,,
4,Jun 28 2022 11:00AM,240614,18,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Jun 28 2022 11:05AM,240607,15,"Brookfield Pharmaceuticals, LLC",42.0,NaN,NaN
3,Jun 28 2022 11:03AM,240606,15,"Alliance Pharma, Inc.",19.0,NaN,NaN
4,Jun 28 2022 11:00AM,240614,18,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Jun 28 2022 11:05AM,240607,15,"Brookfield Pharmaceuticals, LLC",42.0,0.0,0.0
3,Jun 28 2022 11:03AM,240606,15,"Alliance Pharma, Inc.",19.0,0.0,0.0
4,Jun 28 2022 11:00AM,240614,18,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1924780,66.0,4.0,0.0
15,481213,61.0,0.0,0.0
16,481187,5.0,6.0,0.0
17,240550,1.0,0.0,0.0
18,240614,1.0,0.0,0.0
19,2165249,35.0,26.0,24.0
20,240599,15.0,6.0,0.0
21,962222,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1924780,66.0,4.0,0.0
1,15,481213,61.0,0.0,0.0
2,16,481187,5.0,6.0,0.0
3,17,240550,1.0,0.0,0.0
4,18,240614,1.0,0.0,0.0
5,19,2165249,35.0,26.0,24.0
6,20,240599,15.0,6.0,0.0
7,21,962222,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,66.0,4.0,0.0
1,15,61.0,0.0,0.0
2,16,5.0,6.0,0.0
3,17,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,35.0,26.0,24.0
6,20,15.0,6.0,0.0
7,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,66.0
1,15,Released,61.0
2,16,Released,5.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
Executing,4.0,0.0,6.0,0.0,0.0,26.0,6.0,1.0
Released,66.0,61.0,5.0,1.0,1.0,35.0,15.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
1,Executing,4.0,0.0,6.0,0.0,0.0,26.0,6.0,1.0
2,Released,66.0,61.0,5.0,1.0,1.0,35.0,15.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
1,Executing,4.0,0.0,6.0,0.0,0.0,26.0,6.0,1.0
2,Released,66.0,61.0,5.0,1.0,1.0,35.0,15.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()